# Task 1 - Loading the Dataset

In [1]:
import pandas as pd 
df = pd.read_csv('IMDB Dataset.csv')

In [2]:
df.head(10)
df['review'][0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

# Task 2 Transforming Documents into Feature Vectors

In [3]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()

docs = np.array(['The sun is shining','The weather is sweet',
                 'The sun is shining, the weather is sweet, and one and one is two'
    ])

bag= count.fit_transform(docs)
print(count.vocabulary_)
print(bag.toarray())

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}
[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


# Task 3 Term Frequency and Inverse document frequency
#Term frequencies alone do not contribute to distinct information
#tf-idf (t,d) = tf(t,d) X idf(t,d)
#idf(t,d) = log(nd / 1 + df(d,t))
#where nd = total number of documents and df(d,t) = number of documents that contain the term t

In [4]:
from sklearn.feature_extraction.text import TfidfTransformer
fidf = TfidfTransformer(use_idf = True, norm = 'l2', smooth_idf =True)

print(fidf.fit_transform(count.fit_transform(docs)).toarray())

[[0.         0.43370786 0.         0.55847784 0.55847784 0.
  0.43370786 0.         0.        ]
 [0.         0.43370786 0.         0.         0.         0.55847784
  0.43370786 0.         0.55847784]
 [0.50238645 0.44507629 0.50238645 0.19103892 0.19103892 0.19103892
  0.29671753 0.25119322 0.19103892]]


In [5]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()


def stemmer_tokenize(text):
    return [porter.stem(word) for word in text.split()]

stemmer_tokenize('coders like coding and thus they code')
    

['coder', 'like', 'code', 'and', 'thu', 'they', 'code']

# TASK 6 -VECTORISING THE DATASET

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(strip_accents = None,
                       lowercase = False,
                       tokenizer = stemmer_tokenize,
                       use_idf = True,
                       norm = 'l2',
                       smooth_idf = True)
Y = df.sentiment.values
X = tfidf.fit_transform(df.review)

# TASK 7 DOCUMENT CLASSIFICATION USING LOGISTIC REGRESSION

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test,Y_train,Y_test = train_test_split(X,Y,random_state =1, test_size =0.5, shuffle=False)

import pickle
from sklearn.linear_model import LogisticRegressionCV

#model
clf= LogisticRegressionCV(cv=5,
                          scoring='accuracy',
                         random_state=0,
                         n_jobs = -3,
                         verbose = 3,
                         max_iter=300).fit(X_train, Y_train)

#saving the model
saved_model = open('saved_model.sav','wb')

#using the pickle library's dump function to write the trained classifer to the file
pickle.dump(clf, saved_model)
saved_model.close()

[Parallel(n_jobs=-3)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-3)]: Done   2 out of   5 | elapsed:  4.7min remaining:  7.1min
[Parallel(n_jobs=-3)]: Done   5 out of   5 | elapsed:  4.9min finished


# Task 8 Model Evaluation

In [8]:
filename = 'saved_model.sav'

saved_clf = pickle.load(open(filename,'rb'))

#test the saved model on test data
saved_clf.score(X_test, Y_test)


0.8898